In this session, we are going to normalize the grammar and semantic change term
Norm(X) = (X - mean(X)) / std(X)

In [20]:
import pandas as pd
import numpy as np
# read original result 'analyze_semantics_cov_bilstm_512.txt'
# df = pd.read_csv("results/cov/semantics/analyze_semantics_cov_bilstm_512.txt", sep='\t')
df = pd.read_csv("analyze_semantics_d614g_bilstm_512.txt", sep='\t')
df

,pos,wt,mut,prob,change,is_viable,is_escape
0,0,M,A,2.189499e-06,1319.03750,True,False
1,0,M,B,1.117368e-18,1573.24770,False,False
2,0,M,C,1.520572e-06,3205.32890,True,False
3,0,M,D,6.675085e-09,1647.54540,True,False
4,0,M,E,1.990659e-07,1503.97990,True,False
...,...,...,...,...,...,...,...
30547,0,M,V,8.088458e-06,737.07324,True,False
30548,0,M,W,1.014628e-09,1087.49430,True,False
30549,0,M,X,1.909755e-11,395.38600,False,False
30550,0,M,Y,2.780735e-07,554.93100,True,False


In [21]:
# add normalized log(prob) & change
df['log_prob'] = np.log(df['prob'])
df

,pos,wt,mut,prob,change,is_viable,is_escape,log_prob
0,0,M,A,2.189499e-06,1319.03750,True,False,-13.031838
1,0,M,B,1.117368e-18,1573.24770,False,False,-41.335556
2,0,M,C,1.520572e-06,3205.32890,True,False,-13.396424
3,0,M,D,6.675085e-09,1647.54540,True,False,-18.824884
4,0,M,E,1.990659e-07,1503.97990,True,False,-15.429630
...,...,...,...,...,...,...,...,...
30547,0,M,V,8.088458e-06,737.07324,True,False,-11.725072
30548,0,M,W,1.014628e-09,1087.49430,True,False,-20.708744
30549,0,M,X,1.909755e-11,395.38600,False,False,-24.681461
30550,0,M,Y,2.780735e-07,554.93100,True,False,-15.095380


In [22]:
def normalize_series(series, method):
    return method(series)

def min_max_normalize(series):
    return (series - series.min()) / (series.max() - series.min())

def mean_std_normalize(series):
    return (series - series.mean()) / series.std()

df['min_max_log_prob'] = normalize_series(df['log_prob'], min_max_normalize)
df['min_max_change'] = normalize_series(df['change'], min_max_normalize)
df['mean_std_log_prob'] = normalize_series(df['log_prob'], mean_std_normalize)
df['mean_std_change'] = normalize_series(df['change'], mean_std_normalize)
df

,pos,wt,mut,prob,change,is_viable,is_escape,log_prob,min_max_log_prob,min_max_change,mean_std_log_prob,mean_std_change
0,0,M,A,2.189499e-06,1319.03750,True,False,-13.031838,0.724913,0.042126,0.610903,-1.133628
1,0,M,B,1.117368e-18,1573.24770,False,False,-41.335556,0.072818,0.052703,-3.729702,-1.006171
2,0,M,C,1.520572e-06,3205.32890,True,False,-13.396424,0.716513,0.120606,0.554991,-0.187874
3,0,M,D,6.675085e-09,1647.54540,True,False,-18.824884,0.591446,0.055794,-0.277508,-0.968920
4,0,M,E,1.990659e-07,1503.97990,True,False,-15.429630,0.669669,0.049821,0.243182,-1.040901
...,...,...,...,...,...,...,...,...,...,...,...,...
30547,0,M,V,8.088458e-06,737.07324,True,False,-11.725072,0.755019,0.017913,0.811306,-1.425415
30548,0,M,W,1.014628e-09,1087.49430,True,False,-20.708744,0.548043,0.032493,-0.566413,-1.249720
30549,0,M,X,1.909755e-11,395.38600,False,False,-24.681461,0.456515,0.003697,-1.175661,-1.596731
30550,0,M,Y,2.780735e-07,554.93100,True,False,-15.095380,0.677370,0.010335,0.294442,-1.516737


In [23]:
import sys

df['mean_std_sum_rank'] = (df['mean_std_log_prob'] + i * df['mean_std_change']).rank(ascending=False)


def investigate(upper_bound, inc=0.1):
    i = 0
    min_val = (0, sys.maxsize)
    for _ in range(0, upper_bound):
        i += inc
        df['mean_std_sum_rank'] = (df['mean_std_log_prob'] + i * df['mean_std_change']).rank(ascending=False)
        df['min_max_sum_rank'] = (df['min_max_log_prob'] + i * df['min_max_change']).rank(ascending=False)
        df_escape = df[df['is_escape']]
        current_val = df_escape['sum_rank'].mean()
        min_val = (i, current_val) if min_val[1] > current_val else min_val
    return min_val
min_val = investigate(100)
print(f'min value = {min_val}')

min value = (0.30000000000000004, 4495.473684210527)
